<a href="https://colab.research.google.com/github/shu65/theoretical-numerical-linear-algebra/blob/main/%E7%B7%9A%E5%BD%A2%E8%A8%88%E7%AE%97%E3%81%AE%E6%95%B0%E7%90%86_7_2_2_%E3%81%B9%E3%81%8D%E4%B9%97%E6%B3%95%E3%80%80%E9%80%86%E5%8F%8D%E5%BE%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np

a_matrix = np.array([
    [1., 0., 0.,],
    [0., 2., 1.,],
    [0., 1., 1.],
])
a_matrix

array([[1., 0., 0.],
       [0., 2., 1.],
       [0., 1., 1.]])

In [11]:
np.linalg.eig(a_matrix)

(array([0.38196601, 2.61803399, 1.        ]),
 array([[ 0.        ,  0.        ,  1.        ],
        [ 0.52573111, -0.85065081,  0.        ],
        [-0.85065081, -0.52573111,  0.        ]]))

In [12]:
def lu_decompose(a):
  n = a.shape[0]
  t = a.copy()
  for k in range(n):
    w = 1/t[k, k]
    for i in range(k + 1, n):
      t[i, k] = t[i, k]*w
      for j in range(k + 1, n):
        t[i,j] = t[i,j] - t[i, k] * t[k, j]
  l = np.identity(n)
  u = np.zeros_like(a)
  for i in range(n):
    for j in range(n):
      if i > j:
        l[i, j] = t[i, j]
      else:
        u[i, j] = t[i, j]
  return l, u

l, u = lu_decompose(a_matrix)
print("l", l)
print("u", u)

l [[1.  0.  0. ]
 [0.  1.  0. ]
 [0.  0.5 1. ]]
u [[1.  0.  0. ]
 [0.  2.  1. ]
 [0.  0.  0.5]]


In [13]:
np.testing.assert_almost_equal(a_matrix, l @ u)

In [14]:
def l_solve(l, x):
  n = len(x)
  v = np.zeros(n)
  a = l.copy()
  b = x.copy()
  for i in range(n):
    tmp = b[i] / a[i, i]
    v[i] = tmp
    for j in range(i+1, n):
      b[j] -= a[j, i] *  tmp
      a[j, i] = 0
  return v

x = np.ones((a_matrix.shape[1],))/np.sqrt(a_matrix.shape[1]) 
v = l_solve(l, x)
print(l, x)
print(v)

[[1.  0.  0. ]
 [0.  1.  0. ]
 [0.  0.5 1. ]] [0.57735027 0.57735027 0.57735027]
[0.57735027 0.57735027 0.28867513]


In [15]:
np.testing.assert_almost_equal(l @ v, x)

In [16]:
def u_solve(u, v):
  n = len(v)
  y = np.zeros(n)
  a = u.copy()
  b = v.copy()
  for i in reversed(range(n)):
    tmp = b[i] / a[i, i]
    y[i] = tmp
    for j in range(0, i):
      b[j] -= a[j, i] *  tmp
      a[j, i] = 0
    #print(a, b, y)
  return y

x = np.ones((a_matrix.shape[1],))/np.sqrt(a_matrix.shape[1]) 
y = u_solve(u, v)
y

array([0.57735027, 0.        , 0.57735027])

In [17]:
np.testing.assert_almost_equal(u @ y, v)

In [18]:
def inverse_power_method(a, max_iterations=100):
  l, u = lu_decompose(a)
  eigen_vector = np.ones((a.shape[1],))/np.sqrt(a.shape[1])
  print("init eigen_vector", eigen_vector)
  for i in range(max_iterations):
    v = l_solve(l, eigen_vector)
    y = u_solve(u, v)
    inversed_eigen_value = np.linalg.norm(y)
    eigen_vector = y/inversed_eigen_value
    print(f"iter:{i} eigen_value:{1.0/inversed_eigen_value} eigen_vector:{eigen_vector}")
  return 1.0/inversed_eigen_value, eigen_vector

inverse_power_method(a_matrix)

init eigen_vector [0.57735027 0.57735027 0.57735027]
iter:0 eigen_value:1.224744871391589 eigen_vector:[0.70710678 0.         0.70710678]
iter:1 eigen_value:0.5773502691896256 eigen_vector:[ 0.40824829 -0.40824829  0.81649658]
iter:2 eigen_value:0.4140393356054126 eigen_vector:[ 0.16903085 -0.50709255  0.84515425]
iter:3 eigen_value:0.3867462340768809 eigen_vector:[ 0.06537205 -0.52297636  0.84983659]
iter:4 eigen_value:0.38266570437083464 eigen_vector:[ 0.02501564 -0.52532843  0.85053175]
iter:5 eigen_value:0.38206814379405946 eigen_vector:[ 0.00955768 -0.52567235  0.85063343]
iter:6 eigen_value:0.3819809132252592 eigen_vector:[ 0.00365085 -0.52572254  0.85064827]
iter:7 eigen_value:0.3819681854410742 eigen_vector:[ 0.00139451 -0.52572986  0.85065044]
iter:8 eigen_value:0.3819663284607629 eigen_vector:[ 5.32655450e-04 -5.25730930e-01  8.50650754e-01]
iter:9 eigen_value:0.3819660575305264 eigen_vector:[ 2.03456302e-04 -5.25731085e-01  8.50650800e-01]
iter:10 eigen_value:0.3819660180023

(0.38196601125010515,
 array([ 4.90564268e-42, -5.25731112e-01,  8.50650808e-01]))